In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import hdbscan
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
file_path = 'water_potability.csv'  
df = pd.read_csv(file_path)
data = df.copy()
print(data.dtypes)
data=data.dropna()
data = data.reset_index(drop=True)

y = data['Potability']

variable_list = data.columns.tolist()
print(variable_list)
variable_list.remove('Potability')
print(variable_list)

numeric_data = data[variable_list]

# StandardScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_data)
df_scaled = pd.DataFrame(scaled_data, columns=variable_list)

df_scaled['Potability'] = y

unique_counts = df_scaled['Potability'].value_counts()
print("Number of occurrences for each unique value in 'target' column:")
print(unique_counts)

In [ ]:
num_samples_per_class = 700

# Filter rows with diabetes value 0 and 1
diabetes_class_0 = df_scaled[df_scaled['Potability'] == 0].sample(n=num_samples_per_class, random_state=42)
diabetes_class_1 = df_scaled[df_scaled['Potability'] == 1].sample(n=num_samples_per_class, random_state=42)

random_sample = pd.concat([diabetes_class_0, diabetes_class_1], axis=0)
random_sample = random_sample.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
measure = random_sample.copy()
grouped = measure.groupby('Potability')

stats_dict = {}

# Iterate over each group
for group_name, group_df in grouped:
    # Calculate the mean and standard deviation for each column in the group
    mean_values = group_df.mean()
    std_values = group_df.std()
    
    # Store mean and std in a dictionary for each feature
    for feature in group_df.columns:
        if feature not in stats_dict:
            stats_dict[feature] = {}
        
        stats_dict[feature]['mean_' + str(group_name)] = mean_values[feature]
        stats_dict[feature]['std_' + str(group_name)] = std_values[feature]

print(stats_dict)

In [ ]:
feature_to_delete = 'Potability'

if feature_to_delete in stats_dict:
    del stats_dict[feature_to_delete]
    print(f"{feature_to_delete} deleted from stats_dict")
else:
    print(f"{feature_to_delete} does not exist in stats_dict")

In [ ]:
rf_dict = {}

# Iterate over each feature in stats_dict
for feature, stats_values in stats_dict.items():
    # Calculate rf_i value using the f1 formula
    rf_i = ((stats_values['mean_0'] - stats_values['mean_1'])**2) / (stats_values['std_0']**2 + stats_values['std_1']**2)
    
    rf_dict[feature] = rf_i

print(rf_dict)

In [ ]:

max_rf_value = float('-inf')  # Initialize with negative infinity to ensure any calculated value is greater

# Iterate over each feature in rf_dict
for feature, rf_value in rf_dict.items():
    if rf_value > max_rf_value:
        max_rf_value = rf_value

print("Maximum rf value:", max_rf_value)

In [ ]:
f1_overlap = 1 / (1+0.004115415242891468)   # the max rf value
print(f1_overlap)